In [42]:
from pathlib import Path
import sys
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# Path to project root
PROJECT_ROOT = Path.home() / "Desktop" / "Insurance_Project"

# Add root to Python path
sys.path.append(str(PROJECT_ROOT))

In [43]:
from pathlib import Path
import os
import importlib
from src.data import preprocessing as pr
import re
file_path = PROJECT_ROOT/"data"/"raw"/"insurance_claims.csv"
data = pr.load_data(file_path)
# Feature engineering specific to the prototype dataset
data = data.drop('_c39', axis=1)
data.replace('?', np.nan, inplace=True)

for char in ['Y', 'YES']:
    data.replace(char, True, inplace=True)
for char in ['N', 'NO']:
    data.replace(char, False, inplace=True)


C:\Users\phili\AppData\Local\Temp\ipykernel_13152\438751034.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace(char, False, inplace=True)


In [44]:
null_count = int(data.isna().sum().sum()) 
print(f"Null count: {null_count}")

Null count: 972


In [45]:
missing_values = pd.DataFrame(data.isnull().sum().sort_values(ascending=False)) # type: ignore
missing_values

,0
property_damage,360
police_report_available,343
collision_type,178
authorities_contacted,91
months_as_customer,0
age,0
policy_number,0
policy_deductable,0
policy_csl,0
policy_state,0


In [46]:
target = 'total_claim_amount'
X_train, X_test, X_val, y_train, y_test, y_val = pr.split_data(data, target)# type: ignore

In [47]:
data['policy_bind_date'] = pd.to_datetime(data['policy_bind_date'], yearfirst=True)
data['month'] = data['policy_bind_date'].dt.month
data['month_sin'] = np.sin(2 * np.pi * data['month']/12)
data['month_cos'] = np.cos(2 * np.pi * data['month']/12)

data = data.drop('month', axis=1)

In [48]:

data['insured_zip'] = data['insured_zip'].astype(object)
date_cols = data.select_dtypes(include="datetime").columns# type: ignore
bool_cols = []
for col in data.columns:
    unique_vals = set(data[col].dropna().unique())
    if unique_vals.issubset({True, False}):
        data[col] = data[col].astype('boolean')  # nullable bool
        bool_cols.append(col)

for col in bool_cols:
    print(f"Bool Column --------- {col}")
print("\n----------------------------------------------------\n")

for col in date_cols:
    print(f"Date Column --------- {col}")
print("\n----------------------------------------------------\n")
num_cols = data.loc[:,~data.columns.isin(date_cols)].select_dtypes(include="number").columns# type: ignore
for col in num_cols:
    print(f"Numerical Column --------- {col}")

print("\n----------------------------------------------------\n")
cat_cols = data.select_dtypes(include="object").columns# type: ignore
cat_cols = [x for x in cat_cols if x not in bool_cols]
for col in cat_cols:
    print(f"Categorical Column --------- {col}")

num_cols.drop("policy_number")

data = data.drop('policy_bind_date', axis=1)
# Cyclical interpretation of datetime feature

Bool Column --------- property_damage
Bool Column --------- police_report_available
Bool Column --------- fraud_reported

----------------------------------------------------

Date Column --------- policy_bind_date

----------------------------------------------------

Numerical Column --------- months_as_customer
Numerical Column --------- age
Numerical Column --------- policy_number
Numerical Column --------- policy_deductable
Numerical Column --------- policy_annual_premium
Numerical Column --------- umbrella_limit
Numerical Column --------- capital-gains
Numerical Column --------- capital-loss
Numerical Column --------- incident_hour_of_the_day
Numerical Column --------- number_of_vehicles_involved
Numerical Column --------- bodily_injuries
Numerical Column --------- witnesses
Numerical Column --------- total_claim_amount
Numerical Column --------- injury_claim
Numerical Column --------- property_claim
Numerical Column --------- vehicle_claim
Numerical Column --------- auto_year
Nu

In [49]:
print(X_train)

     months_as_customer  age  policy_number policy_bind_date policy_state  \
541                 239   41         743092       2013-11-11           OH   
440                 108   31         492224       2005-12-09           IN   
482                 116   30         996253       2001-11-29           IN   
422                   8   21         355085       2012-10-09           IN   
778                 161   38         192524       2004-01-02           IL   
..                  ...  ...            ...              ...          ...   
106                 464   61         632627       1990-10-07           OH   
270                 369   55         577810       2013-04-15           OH   
860                 230   42         175960       2004-11-16           IN   
435                 102   28         810189       1999-08-29           OH   
102                 279   41         389238       2001-06-06           IL   

    policy_csl  policy_deductable  policy_annual_premium  umbrella_limit  \

In [50]:
# data = data.drop('policy_number', axis=1)
# data = data.drop('incident_city', axis=1)
# data = data.drop('incident_location', axis=1)
# data = data.drop('insured_zip', axis=1)
# data = data.drop('insured_hobbies', axis=1)
# data = data.drop('injury_claim', axis=1)
# data = data.drop('property_claim', axis=1)
# data = data.drop('vehicle_claim', axis=1)
# data = data.drop('incident_date', axis=1)




In [51]:

# numerical_columns = X_train.select_dtypes(include='number').columns# type: ignore
# ordinal_columns = ['policy_state','policy_csl','insured_sex', 'insured_education_level', 'incident_type','incident_severity',]
# ordinal_categories = [['OH', 'IN', 'IL'],
#                       ['100/300', '250/500', '500/1000'],
#                       ['MALE', 'FEMALE'],
#                       ['High School', 'Masters','JD', 'MD'],
#                       ['Single Vehicle Collision', 'Multi-vehicle Collision'],
#                       ['Minor Damage', 'Major Damage', 'Total Loss']]
# nominal_columns = [x for x in cat_cols if x not in ordinal_columns]


In [52]:
data = data.drop('policy_number', axis=1)
data = data.drop('insured_hobbies', axis=1)
data = data.drop('injury_claim', axis=1)
data = data.drop('property_claim', axis=1)
data = data.drop('vehicle_claim', axis=1)
data = data.drop('incident_date', axis=1)
data = data.drop('collision_type', axis=1)
data = data.drop('number_of_vehicles_involved', axis=1)
data = data.drop('incident_severity', axis=1)
data = data.drop('incident_type', axis=1)
data = data.drop('bodily_injuries', axis=1)
data = data.drop('property_damage', axis=1)
data = data.drop('police_report_available', axis=1)
data = data.drop('authorities_contacted', axis=1)
data = data.drop('incident_hour_of_the_day', axis=1)
data = data.drop('fraud_reported', axis=1)
data = data.drop('capital-gains', axis=1)
data = data.drop('capital-loss', axis=1)



In [53]:
numerical_columns = X_train.select_dtypes(include='number').columns# type: ignore
ordinal_columns = ['policy_state','policy_csl','insured_sex', 'insured_education_level']
ordinal_categories = [['OH', 'IN', 'IL'],
                      ['100/300', '250/500', '500/1000'],
                      ['MALE', 'FEMALE'],
                      ['High School', 'Masters','JD', 'MD'],]
nominal_columns = [x for x in cat_cols if x not in ordinal_columns]


In [54]:
print(numerical_columns)

Index(['months_as_customer', 'age', 'policy_number', 'policy_deductable',
       'policy_annual_premium', 'umbrella_limit', 'insured_zip',
       'capital-gains', 'capital-loss', 'incident_hour_of_the_day',
       'number_of_vehicles_involved', 'bodily_injuries', 'witnesses',
       'injury_claim', 'property_claim', 'vehicle_claim', 'auto_year'],
      dtype='object')


In [55]:
importlib.reload(pr)
preprocessor, X_train_processed = pr.prepreprocessing_pipeline(
    X = X_train,
    numerical_cols=numerical_columns,
    ordinal_cols=ordinal_columns,
    ordinal_categories=ordinal_categories,
    nominal_cols=nominal_columns,
    bool_cols=bool_cols
)

In [56]:
importlib.reload(pr)
global_mean = y_train.mean()
X_val_processed = pr.apply_preprocessing_pipeline(X_val,
                                                  preprocessor=preprocessor)

X_test_processed = pr.apply_preprocessing_pipeline(X_test,
                                                  preprocessor=preprocessor)

In [57]:
importlib.reload(pr)
X_train_processed, y_train = pr.remove_null(X_train_processed, y_train)
X_val_processed, y_val = pr.remove_null(X_val_processed, y_val)
X_test_processed, y_test = pr.remove_null(X_test_processed, y_test)

In [58]:
train_data = pd.DataFrame(X_train_processed)
test_data = pd.DataFrame(X_test_processed)
val_data = pd.DataFrame(X_val_processed)
processed_data = pd.concat([
    train_data,
    test_data,
    val_data
], ignore_index=True)   

In [59]:
processed_data.shape

(1000, 38)

In [60]:
print(processed_data.isnull().sum().sum())

0


In [61]:
i = 0
for df in [train_data, test_data, val_data]:
    print(f"Null values in {i}: {df.isnull().sum().sum()}")
    i += 1

Null values in 0: 0
Null values in 1: 0
Null values in 2: 0


In [62]:
y_train_data = pd.Series(y_train)
y_test_data = pd.Series(y_test)
y_val_data = pd.Series(y_val)

In [63]:
y_train_data

541    10790
440    57330
482    59670
422    91260
778    60480
       ...  
106    79800
270    85300
860    58850
435    73400
102    28800
Name: total_claim_amount, Length: 700, dtype: int64

In [64]:
data_folder = PROJECT_ROOT/"data"/"processed"
train_data.to_csv(data_folder/"X_train.csv", index=False)
test_data.to_csv(data_folder/"X_test.csv", index=False)
val_data.to_csv(data_folder/"X_val.csv",index=False)
y_train_data.to_csv(data_folder/"y_train.csv",index=False)
y_test_data.to_csv(data_folder/"y_test.csv",index=False)
y_val_data.to_csv(data_folder/"y_val.csv",index=False)

In [65]:
# numerical_columns = X_train.select_dtypes(include='number').columns# type: ignore
# ordinal_columns = ['policy_state','policy_csl','insured_sex', 'insured_education_level', 'incident_type','incident_severity',]
# ordinal_categories = [['OH', 'IN', 'IL'],
#                       ['100/300', '250/500', '500/1000'],
#                       ['MALE', 'FEMALE'],
#                       ['High School', 'Masters','JD', 'MD'],
#                       ['Single Vehicle Collision', 'Multi-vehicle Collision'],
#                       ['Minor Damage', 'Major Damage', 'Total Loss']]
# nominal_columns = [x for x in cat_cols if x not in ordinal_columns]


In [66]:
numerical_columns = X_train.select_dtypes(include='number').columns# type: ignore
ordinal_columns = ['policy_state','policy_csl','insured_sex', 'insured_education_level']
ordinal_categories = [['OH', 'IN', 'IL'],
                      ['100/300', '250/500', '500/1000'],
                      ['MALE', 'FEMALE'],
                      ['High School', 'Masters','JD', 'MD'],]
nominal_columns = [x for x in cat_cols if x not in ordinal_columns]

In [67]:
numerical_columns = [col for col in numerical_columns if col in data.columns]
ordinal_columns = [col for col in ordinal_columns if col in data.columns]
nominal_columns = [col for col in nominal_columns if col in data.columns]
bool_cols = [col for col in bool_cols if col in data.columns]

In [68]:
print(numerical_columns)

['months_as_customer', 'age', 'policy_deductable', 'policy_annual_premium', 'umbrella_limit', 'insured_zip', 'witnesses', 'auto_year']


In [69]:
from sklearn.model_selection import train_test_split
target = 'total_claim_amount'
X, y = data[[col for col in data.columns if col not in target]], data[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

preprocessor, X_train_processed = pr.prepreprocessing_pipeline(
    X = X_train,
    numerical_cols=numerical_columns,
    ordinal_cols=ordinal_columns,
    ordinal_categories=ordinal_categories,
    nominal_cols=nominal_columns,
    bool_cols=bool_cols
)
X_test_processed = pr.apply_preprocessing_pipeline(X_test, preprocessor)
train_reg, test_reg = pd.DataFrame(X_train_processed), pd.DataFrame(X_test_processed)
y_train_reg, y_test_reg = pd.Series(y_train), pd.Series(y_test)

train_reg.to_csv(data_folder/"reg"/"X_train.csv", index=False)
test_reg.to_csv(data_folder/"reg"/"X_test.csv", index=False)
y_train_reg.to_csv(data_folder/"reg"/"y_train.csv", index=False)
y_test_reg.to_csv(data_folder/"reg"/"y_test.csv", index=False)


In [70]:
print(X.shape)

(1000, 21)


In [71]:
features = X.columns
features

Index(['months_as_customer', 'age', 'policy_state', 'policy_csl',
       'policy_deductable', 'policy_annual_premium', 'umbrella_limit',
       'insured_zip', 'insured_sex', 'insured_education_level',
       'insured_occupation', 'insured_relationship', 'incident_state',
       'incident_city', 'incident_location', 'witnesses', 'auto_make',
       'auto_model', 'auto_year', 'month_sin', 'month_cos'],
      dtype='object')

In [72]:
X_train = pd.DataFrame(X_train_processed)

In [73]:
X_test = pd.DataFrame(X_test_processed)

In [74]:
X_test

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,-0.029730,0.118133,-0.226423,0.768863,-0.486554,-0.798464,-1.340806,1.650994,2.0,0.0,1.0,0.0,0.0,68.0,158.0,98.0,117.0,0.0,63.0,22.0
1,0.075233,0.008071,-0.226423,0.012708,-0.486554,-0.668734,1.366880,0.486369,0.0,0.0,1.0,2.0,0.0,53.0,157.0,232.0,117.0,0.0,72.0,38.0
2,2.148248,1.989177,1.403822,0.774370,-0.486554,-0.832124,-0.438244,0.486369,0.0,1.0,0.0,-1.0,0.0,71.0,169.0,219.0,117.0,0.0,63.0,29.0
3,-1.604171,-0.212052,-1.041545,0.944760,-0.486554,1.546968,-0.438244,-0.345505,2.0,1.0,1.0,-1.0,0.0,65.0,150.0,232.0,132.0,0.0,68.0,25.0
4,-0.362112,-0.432175,1.403822,-0.524273,-0.486554,-0.768659,1.366880,-0.678255,2.0,2.0,1.0,1.0,0.0,51.0,150.0,198.0,132.0,0.0,59.0,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.486337,0.558378,1.403822,1.658420,-0.486554,-0.663838,-1.340806,-0.511880,0.0,1.0,1.0,1.0,0.0,71.0,150.0,232.0,134.0,0.0,65.0,34.0
96,0.232677,0.008071,-0.226423,-1.428378,2.545385,-0.663824,1.366880,-0.511880,2.0,0.0,1.0,3.0,0.0,71.0,131.0,219.0,117.0,0.0,62.0,19.0
97,0.687516,0.778501,1.403822,1.260061,-0.486554,-0.715555,0.464318,1.151869,0.0,0.0,0.0,3.0,0.0,62.0,135.0,232.0,142.0,0.0,64.0,13.0
98,0.871201,1.218747,-1.041545,-1.886822,-0.486554,1.643121,1.366880,0.486369,0.0,0.0,0.0,3.0,0.0,71.0,169.0,198.0,123.0,0.0,72.0,34.0


In [75]:
X_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.827466,0.888563,1.403822,0.354435,-0.486554,1.616215,-1.340806,-0.844630,2.0,2.0,1.0,-1.0,1.0,87.0,158.0,232.0,117.0,1.0,74.0,24.0
1,0.748744,0.448317,-0.226423,0.555854,-0.486554,1.483920,1.366880,0.819119,0.0,2.0,1.0,-1.0,1.0,68.0,157.0,232.0,117.0,1.0,61.0,39.0
2,0.635035,0.778501,1.403822,-0.665525,-0.486554,-0.927500,0.464318,0.319995,0.0,0.0,0.0,-1.0,1.0,51.0,150.0,219.0,135.0,1.0,63.0,34.0
3,2.034538,2.209300,-0.226423,0.354640,-0.486554,-0.998759,0.464318,1.484619,0.0,2.0,0.0,-1.0,1.0,53.0,169.0,198.0,123.0,1.0,59.0,17.0
4,1.710903,1.769054,1.403822,-1.054966,-0.486554,-0.523262,-0.438244,0.819119,2.0,2.0,0.0,2.0,1.0,71.0,157.0,232.0,134.0,1.0,66.0,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,-1.709134,-1.973035,-1.041545,-0.946715,-0.486554,-0.529198,-1.340806,1.151869,1.0,2.0,0.0,0.0,1.0,51.0,150.0,198.0,135.0,1.0,59.0,17.0
896,0.092727,-0.322113,1.403822,-1.088419,2.545385,-0.568295,1.366880,0.153620,2.0,0.0,1.0,-1.0,1.0,87.0,169.0,232.0,142.0,1.0,74.0,24.0
897,-0.711988,-1.092543,-0.226423,1.552182,-0.486554,1.632511,-0.438244,1.484619,0.0,1.0,1.0,1.0,1.0,63.0,158.0,219.0,117.0,1.0,72.0,34.0
898,0.425109,0.778501,1.403822,-0.762597,-0.486554,-0.402533,0.464318,1.484619,1.0,2.0,1.0,2.0,1.0,51.0,169.0,98.0,134.0,1.0,65.0,34.0


In [76]:
importlib.reload(pr)
processed_data = pr.clean_data(data=data)

In [77]:
print(processed_data.isna().sum().sum())

0


In [78]:
processed_data.to_csv(PROJECT_ROOT/"dashboard"/"assets"/"dashboard_data.csv")

In [79]:
processed_data

,months_as_customer,age,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,insured_sex,insured_education_level,...,incident_state,incident_city,incident_location,witnesses,total_claim_amount,auto_make,auto_model,auto_year,month_sin,month_cos
0,328.0,48.0,OH,250/500,1000.0,1406.91,0.0,466132,MALE,MD,...,SC,Columbus,9935 4th Drive,2.0,71610.0,Saab,92x,2004.0,-8.660254e-01,5.000000e-01
1,228.0,42.0,IN,250/500,2000.0,1197.22,5000000.0,468176,MALE,MD,...,VA,Riverwood,6608 MLK Hwy,0.0,5070.0,Mercedes,E400,2007.0,1.224647e-16,-1.000000e+00
2,134.0,29.0,OH,100/300,2000.0,1413.14,5000000.0,430632,FEMALE,PhD,...,NY,Columbus,7121 Francis Lane,3.0,34650.0,Dodge,RAM,2007.0,-1.000000e+00,-1.836970e-16
3,256.0,41.0,IL,250/500,2000.0,1415.74,6000000.0,608117,FEMALE,PhD,...,OH,Arlington,6956 Maple Drive,2.0,63400.0,Chevrolet,Tahoe,2014.0,5.000000e-01,-8.660254e-01
4,228.0,44.0,IL,500/1000,1000.0,1583.91,6000000.0,610706,MALE,Associate,...,NY,Arlington,3041 3rd Ave,1.0,6500.0,Accura,RSX,2009.0,1.224647e-16,-1.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,3.0,38.0,OH,500/1000,1000.0,1310.80,0.0,431289,FEMALE,Masters,...,NC,Northbrook,6045 Andromedia St,1.0,87200.0,Honda,Accord,2006.0,-5.000000e-01,-8.660254e-01
996,285.0,41.0,IL,100/300,1000.0,1436.79,0.0,608177,FEMALE,PhD,...,SC,Northbend,3092 Texas Drive,3.0,108480.0,Volkswagen,Passat,2015.0,5.000000e-01,8.660254e-01
997,130.0,34.0,OH,250/500,500.0,1383.49,3000000.0,442797,FEMALE,Masters,...,NC,Arlington,7629 5th St,3.0,67500.0,Suburu,Impreza,1996.0,8.660254e-01,5.000000e-01
998,458.0,62.0,IL,500/1000,2000.0,1356.92,5000000.0,441714,MALE,Associate,...,NY,Arlington,6128 Elm Lane,1.0,46980.0,Audi,A5,1998.0,-5.000000e-01,8.660254e-01


In [80]:
importlib.reload(pr)

<module 'src.data.preprocessing' from 'C:\\Users\\phili\\Desktop\\Insurance_Project\\src\\data\\preprocessing.py'>

In [81]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

preprocessor_pickle = pr.unfit_prepreprocessing_pipeline(
    numerical_cols=numerical_columns,
    ordinal_cols=ordinal_columns,
    ordinal_categories=ordinal_categories,
    nominal_cols=nominal_columns,
    bool_cols=bool_cols)
preprocessor_pickle.fit(X)

,transformers,"[('num', ...), ('ord', ...), ...]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,missing_values,nan
,strategy,'median'
,fill_value,None


In [82]:
import joblib
joblib.dump(preprocessor_pickle, PROJECT_ROOT/"models"/"preprocessor.pkl")
joblib.dump(features, PROJECT_ROOT/"models"/"features.pkl")


['C:\\Users\\phili\\Desktop\\Insurance_Project\\models\\features.pkl']